2.Write a function that lists and counts the number of divisors for an input value.
Example 1:
Input: 5
Output: “There are 2 divisors: 1 and 5”
Example 2:
Input: 40
Output: “There are 8 divisors: 1, 2, 4, 5, 8, 10, 20, and 40”

In [210]:
def fact(num):
    str_t=''
    sum=1
    add_string='and'+str(num)
    for i in range(1,num+1):
        if num%i==0 and num!=i:
            str_t=str_t+str(i)+','
            sum=sum+1
    str_t=str_t.rstrip(str_t[-1])
    str_t = "".join((str_t, add_string))
    print('Total {} divisors : {} '.format(sum,str_t))

In [211]:
fact(40)

Total 8 divisors : 1,2,4,5,8,10,20and40 


In [212]:
fact(5)

Total 2 divisors : 1and5 


1.Look up the Adam optimization functions in PyTorch
https://pytorch.org/docs/stable/optim.html . How does it work? Try at least one other
optimization function with the diabetes dataset shown in class. How does the model
perform with the new optimizer? Did it perform better or worse than Adam? Why do you
think that is?

In [196]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [197]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [198]:
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8514, -0.9801, -0.4048,  ..., -0.6077,  0.3108, -0.7922],
        [ 0.3566,  0.1614,  0.4654,  ..., -0.3021, -0.1164,  0.5610],
        [-0.5494, -0.5045, -0.6223,  ...,  0.3726, -0.7649, -0.7076],
        ...,
        [-0.8514, -0.7582,  0.0303,  ...,  0.7800, -0.7861, -0.2847],
        [ 1.8665, -0.3142,  0.0303,  ..., -0.5695, -1.0194,  0.5610],
        [ 0.0546,  0.7322, -0.6223,  ..., -0.3149, -0.5770,  0.3073]])


In [199]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x


In [221]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [215]:
for i in list_op:
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr = 0.01)

In [216]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch

Epoch number: 1 with loss: 0.6474167108535767
Epoch number: 11 with loss: 0.5238224267959595
Epoch number: 21 with loss: 0.45228567719459534
Epoch number: 31 with loss: 0.41647642850875854
Epoch number: 41 with loss: 0.39122122526168823
Epoch number: 51 with loss: 0.3679707646369934
Epoch number: 61 with loss: 0.3427005112171173
Epoch number: 71 with loss: 0.3138033151626587
Epoch number: 81 with loss: 0.28234556317329407
Epoch number: 91 with loss: 0.2524297833442688
Epoch number: 101 with loss: 0.22721914947032928
Epoch number: 111 with loss: 0.20655877888202667
Epoch number: 121 with loss: 0.18467789888381958
Epoch number: 131 with loss: 0.16674776375293732
Epoch number: 141 with loss: 0.14951622486114502
Epoch number: 151 with loss: 0.13775937259197235
Epoch number: 161 with loss: 0.12599729001522064
Epoch number: 171 with loss: 0.11742447316646576
Epoch number: 181 with loss: 0.1088448092341423
Epoch number: 191 with loss: 0.10019753128290176
Epoch number: 201 with loss: 0.0948677

In [217]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())

In [218]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [219]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.7337662337662337


In [220]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.74      0.78       100
           1       0.60      0.72      0.66        54

    accuracy                           0.73       154
   macro avg       0.72      0.73      0.72       154
weighted avg       0.75      0.73      0.74       154



|Model.....| Accuracy values.....|    
|Adamdelta | 0.6493506493506493  |
|Adagrad   | 0.7012987012987013  | 
|Adam      | 0.6948051948051948  |
|AdamW     | 0.7337662337662337  |
|Sparseadam| 0.6493506493506493  |
|Adamax    | 0.7012987012987013  |
|ASGD      | 0.6493506493506493  |
|Rmsprop   | 0.7272727272727273  |
|Rprop     | 0.7142857142857143  |
|RGD       | 0.6493506493506493  |

#### When compared other optimizers with Adam we can observe that AdamW, Rprop and Rmsprop has got more accuracy. 